In [1]:
# 1. Import thư viện cần thiết
from bs4 import BeautifulSoup
import json
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [2]:
retry_strategy = Retry(
    total=3,  # Số lần thử lại tối đa
    backoff_factor=1,  # Thời gian chờ giữa các lần thử lại (1 giây, 2 giây, 4 giây,...)
    status_forcelist=[429, 500, 502, 503, 504],  # Mã trạng thái yêu cầu retry
    allowed_methods=["HEAD", "GET", "OPTIONS"]  # Chỉ áp dụng retry với các phương thức này
)

adapter = HTTPAdapter(max_retries=retry_strategy)
http = requests.Session()
http.mount("https://", adapter)

In [3]:
link = 'https://medlatec.vn/tu-dien-benh-ly/?Alphabet=&Search='
response = requests.get(link)
soup = BeautifulSoup(response.text, 'html.parser')
disease = soup.find('ul', class_='disease-list')

In [4]:
disease_list = [ f'https://medlatec.vn{i['href']}' for i in disease.find_all('a')]

In [12]:
len(disease_list)

643

In [5]:
def parse_html(link, i=0):
    try:
        response = http.get(link, timeout=20)
        soup = BeautifulSoup(response.content, "html.parser")


        title = soup.find('h1',class_='page-title').text.strip().replace('\n', '').replace('\xa0', ' ').replace('\r', '').replace('  ', ' ')

        # Lấy nội dung của từng phần
        html_content = soup.find('div', class_='description')
        if not html_content:
            return None

        # Hàm trích xuất nội dung của từng phần
        def extract_section_content(tag):
            content = []
            for sibling in tag.find_next():
                if sibling.name in ['h2', 'h3', 'h4', 'h5', 'h6']:
                    break  # Kết thúc khi gặp tiêu đề mới
                if sibling.name == 'p':
                    # Kiểm tra nếu thẻ <p> có chứa <strong>
                    if sibling.find('strong'):
                        # Thực hiện hành động đặc biệt khi tìm thấy <strong> trong <p>
                        strong_content = sibling.find('strong').text.strip()
                        content.append({ 'strong_text': strong_content})
                    else:
                        content.append(sibling.text.strip())
                elif sibling.name in ['ul', 'ol']:
                    list_items = [li.text.strip() for li in sibling.find_all('li')]
                    list_type = 'list' if sibling.name == 'ul' else 'ordered_list'
                    content.append({list_type: list_items})
                elif sibling.name == 'blockquote':
                    content.append({'quote': sibling.text.strip()})
            return content


        # Trích xuất các mục chính và lồng nhau
        sections = []
        current_hierarchy = {2: None, 3: None, 4: None, 5: None, 6: None}

        for section in html_content.find_all(['h2', 'h3', 'h4', 'h5', 'h6']):
            level = int(section.name[1])  # Lấy số từ tên thẻ (h2 -> 2, h3 -> 3,...)
            section_data = {
                'heading': section.text.strip(),
                'content': extract_section_content(section)
            }

            if level == 2:
                sections.append(section_data)
                current_hierarchy[2] = sections[-1]
            else:
                parent_section = current_hierarchy[level - 1]
                if 'subsections' not in parent_section:
                    parent_section['subsections'] = []
                parent_section['subsections'].append(section_data)
            current_hierarchy[level] = section_data

        # Chuyển đổi thành JSON
        data = {
            "title": title,
            "sections": sections,
            "source_link": link,
            "domain": 'Bệnh lí'
        }

        # Lưu vào file JSON
        with open(f'data/article_data{i}.json', 'w', encoding='utf-8') as json_file:
            json.dump(data, json_file, ensure_ascii=False, indent=4)

        return True
    except:
        return False

In [14]:
i = 1
j = 1
miss = []
for link in disease_list:
    print(j)
    j += 1
    try:
        tmp = parse_html(link, i)
        if tmp:  # Nếu tmp == True, có nghĩa là parse_html đã thành công
            i += 1
        else:
            miss.append(link)  # Thêm vào miss nếu parse_html trả về None
    except Exception as e:
        miss.append(link)  # Thêm vào miss nếu có ngoại lệ
        print(f"Lỗi khi xử lý {link}: {e}")  # In chi tiết lỗi để kiểm tra


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [23]:
l = 'https://medlatec.vn/tu-dien-benh-ly/hoi-chung-tiet-dich-nieu-dao-do-lau-sejhm'

response = http.get(l, timeout=20)
soup = BeautifulSoup(response.content, "html.parser")


title = soup.find('h1',class_='page-title').text.strip().replace('\n', '').replace('\xa0', ' ').replace('\r', '').replace('  ', ' ')

# Lấy nội dung của từng phần
html_content = soup.find('div', class_='description')


# Hàm trích xuất nội dung của từng phần
def extract_section_content(tag):
    content = []
    for sibling in tag.find_next():
        if sibling.name in ['h2', 'h3', 'h4', 'h5', 'h6']:
            break  # Kết thúc khi gặp tiêu đề mới
        if sibling.name == 'p':
            # Kiểm tra nếu thẻ <p> có chứa <strong>
            if sibling.find('strong'):
                # Thực hiện hành động đặc biệt khi tìm thấy <strong> trong <p>
                strong_content = sibling.find('strong').text.strip()
                content.append({ 'strong_text': strong_content})
            else:
                content.append(sibling.text.strip())
        elif sibling.name in ['ul', 'ol']:
            list_items = [li.text.strip() for li in sibling.find_all('li')]
            list_type = 'list' if sibling.name == 'ul' else 'ordered_list'
            content.append({list_type: list_items})
        elif sibling.name == 'blockquote':
            content.append({'quote': sibling.text.strip()})
    return content


# Trích xuất các mục chính và lồng nhau
sections = []
current_hierarchy = {2: None, 3: None, 4: None, 5: None, 6: None}

for section in html_content.find_all(['h2', 'h3', 'h4', 'h5', 'h6']):
    level = int(section.name[1])  # Lấy số từ tên thẻ (h2 -> 2, h3 -> 3,...)
    section_data = {
        'heading': section.text.strip(),
        'content': extract_section_content(section)
    }

    if level == 2:
        sections.append(section_data)
        current_hierarchy[2] = sections[-1]
    else:
        parent_section = current_hierarchy[level - 1]
        if 'subsections' not in parent_section:
            parent_section['subsections'] = []
        parent_section['subsections'].append(section_data)
    current_hierarchy[level] = section_data
    print(sections)
# Chuyển đổi thành JSON
data = {
    "title": title,
    "sections": sections,
    "source_link": link,
    "domain": 'Bệnh lí'
}


[{'heading': 'Tổng quan Hội chứng tiết dịch niệu đạo do lậu', 'content': ['Niệu đạo là ống nối giữa bàng quang và lỗ sáo - lỗ đái, niệu đạo giúp nước tiểu đào thải từ bang quang ra ngoài cơ thẻ. Ở nam giới, niệu đạo còn có tác dụng phóng tinh dịch từ túi tinh ra khỏi cơ thể, gọi là hiện tượng xuất tinh.', 'Nhóm các triệu chứng bao gồm hiện tượng chảy dịch, chảy mủ từ trong lỗ niệu đạo của nam giới, tiểu buốt rắt, tiểu khó được gọi là hội chứng tiết dịch niệu đạo. Nguyên nhân tiết dịch thường do viêm nhiễm và nếu không được điều trị sớm có thể dẫn tới các biến chứng như chít hẹp niệu đạo, viêm tinh hoàn hoặc ảnh hưởng tới chức năng sinh sản, vô sinh.', '', 'Hội chứng tiết dịch niệu đạo do lậu', {'strong_text': 'Dịch tễ'}, 'Theo thống kê trên thế giới mỗi năm có 62 triệu ca mặc mới lậu cầu, riêng khu vực Đông Á và Đông Nam Á là 29 triệu ca. Theo báo cáo hàng năm, Việt Nam có khoảng hơn 3.000 ca, còn tính cả những con số ca bệnh lậu không được thống kê cho không được chẩn đoán đúng hoặc n

In [15]:
miss

['https://medlatec.vn/tu-dien-benh-ly/benh-sot-ret-sqlwu',
 'https://medlatec.vn/tu-dien-benh-ly/viem-to-chuc-hoc-mat-shwia',
 'https://medlatec.vn/tu-dien-benh-ly/ap-xe-than-sgkuc',
 'https://medlatec.vn/tu-dien-benh-ly/bat-tuong-xung-dau-chau-stiqi',
 'https://medlatec.vn/tu-dien-benh-ly/lao-dong-nhiem-hiv-smbet',
 'https://medlatec.vn/tu-dien-benh-ly/bong-vong-mac-syslj',
 'https://medlatec.vn/tu-dien-benh-ly/hoi-chung-tiet-dich-nieu-dao-do-lau-sejhm',
 'https://medlatec.vn/tu-dien-benh-ly/tac-dong-mach-vong-mac-soclk',
 'https://medlatec.vn/tu-dien-benh-ly/tac-tinh-mach-vong-mac-sffiy',
 'https://medlatec.vn/tu-dien-benh-ly/lao-dong-nhiem-hiv-sptki',
 'https://medlatec.vn/tu-dien-benh-ly/lao-dong-nhiem-hiv-szskh',
 'https://medlatec.vn/tu-dien-benh-ly/lao-dong-nhiem-hiv-shuxm']

In [24]:
import json

# Đọc dữ liệu từ file JSON
for i in range(1, 632):
    with open(f'data/article_data{i}.json', 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    # Xử lý: xóa các dòng trống trong mỗi 'content' của từng 'section'
    for section in data['sections']:
        section['content'] = [line for line in section['content'] if not isinstance(line, str) or line.strip()]

    # Ghi dữ liệu đã xử lý trở lại file JSON
    with open(f'data/article_data{i}.json', 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

print("Đã xóa các dòng trống trong content.")


Đã xóa các dòng trống trong content.
